In [ ]:
# default_exp utils

# Utils 

This notebook contains useful functions for the rest of the scripts. 

In [ ]:
#export
import GPUtil
import torch
import time 
import torchsnooper
from timeit import default_timer as timer
from threading import Thread

In [ ]:
#hide 
#from nbdev import show_doc

## Code timing

In [ ]:
#export
class timecode:
    """This class is used for timing code"""
    def __enter__(self):
        self.t0 = timer()
        return self

    def __exit__(self, type, value, traceback):
        self.t = timer() - self.t0

In [ ]:
with timecode() as t: 
    time.sleep(1)
print(t.t)

with timecode() as t: 
    x = [0,1,2,3,4]
    y = [o+1 for o in x]
print(t.t)

1.001036524772644
2.946704626083374e-06


## GPU profiling

In [ ]:
#export
def print_device_info(): 
    """
    Prints some statistics around versions and the GPU's available for
    the host machine
    """
    import torch
    import sys
    print("######## Diagnostics and version information ######## ")
    print('__Python VERSION:', sys.version)
    print('__pyTorch VERSION:', torch.__version__)
    print('__CUDA VERSION', )
    from subprocess import call
    # call(["nvcc", "--version"]) does not work
    #! nvcc --version
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__Devices')
    call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
    print('Active CUDA Device: GPU', torch.cuda.current_device())
    print ('Available devices ', torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name())
    print ('Current cuda device ', torch.cuda.current_device())
    print("#################################################################")
    

In [ ]:
#export    
def dump_tensors(gpu_only=True):
    """Prints a list of the Tensors being tracked by the garbage collector. 
    Useful when running into an out of memory error on the GPU. """
    import gc
    total_size = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if not gpu_only or obj.is_cuda:
                    print("%s:%s%s %s" % (type(obj).__name__, 
                                            " GPU" if obj.is_cuda else "",
                                            " pinned" if obj.is_pinned else "",
                                            pretty_size(obj.size())))
                    total_size += obj.numel()
            elif hasattr(obj, "data") and torch.is_tensor(obj.data):
                if not gpu_only or obj.is_cuda:
                    print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
                                                    type(obj.data).__name__, 
                                                    " GPU" if obj.is_cuda else "",
                                                    " pinned" if obj.data.is_pinned else "",
                                                    " grad" if obj.requires_grad else "", 
                                                    " volatile" if obj.volatile else "",
                                                    pretty_size(obj.data.size())))
                    total_size += obj.data.numel()
        except Exception as e:
            pass        
    print("Total size:", total_size)   
    

In [ ]:
#export      
class Monitor(Thread):
    """Use this to check that you are using the GPU during your pytorch functions and to track memory usage 
    of the GPU's as well.""" 
    def __init__(self, delay):
        super(Monitor, self).__init__()
        self.stopped = False
        self.delay = delay # Time between calls to GPUtil
        self.start()

    def run(self):
        while not self.stopped:
            GPUtil.showUtilization()
            time.sleep(self.delay)

    def stop(self):
        self.stopped = True
        

In [ ]:
#export
def show_gpu(msg):
    """
    ref: https://github.com/huggingface/transformers/issues/1742#issue-518262673
    put in logger.info()
    """
    import subprocess
    def query(field):
        return(subprocess.check_output(
            ['nvidia-smi', f'--query-gpu={field}',
                '--format=csv,nounits,noheader'], 
            encoding='utf-8'))
    def to_int(result):
        return int(result.strip().split('\n')[0])
    
    used = to_int(query('memory.used'))
    total = to_int(query('memory.total'))
    pct = used/total
    return f"{msg} {100*pct:2.1f}% ({used} out of {total})" 

## Tensor utils 

In [ ]:
#export
def round_t(t, dp=2):
    """Return rounded tensors for easy viewing. t is a tensor, dp=decimal places"""
    if t.device.type == "cuda": t=t.cpu()
    return t.detach().numpy().round(dp)     

In [ ]:
round_t(torch.rand((10,2)))

array([[0.63, 0.99],
       [0.35, 0.64],
       [0.94, 0.83],
       [0.86, 0.16],
       [0.92, 0.41],
       [0.01, 0.72],
       [0.57, 0.63],
       [0.11, 0.33],
       [0.19, 0.57],
       [0.26, 0.95]], dtype=float32)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 10_data_cleaning.ipynb.
Converted 30_logging.ipynb.
Converted index.ipynb.
